# Mixed-keV analyses
This notebook will demonstrate how it is possible (albeit not necessarily advised) to perform analyses with standards and unknowns collected at different beam energies. 

Here, we take our favorite SRM glass, K412, and use data from 15 keV and 20 keV.

In [ ]:
using NeXLSpectrum, DataFrames

path = joinpath(@__DIR__,"..","test")

#### Example at 20 keV

In [ ]:
al2o3 = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "20 keV", "IIIE Al2O3[$i][4].msa") for i in 0:4)))
sio2 = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "20 keV", "IIIE SiO2[$i][4].msa") for i in 0:4)))
fe = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "20 keV", "IIIE Fe[$i][4].msa") for i in 0:4)))
caf2 = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "20 keV", "IIIE CaF2[$i][4].msa") for i in 0:4)))
mgo = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "20 keV", "IIIE MgO[$i][4].msa") for i in 0:4)))
unk20 = loadspectrum.(joinpath(path,"Multi-keV", "20 keV", "IIIE K412[$i][4].msa") for i in 0:4)
    
refs20 = references( [
    reference( n"Al", al2o3 ),
    reference( n"Si", sio2 ),
    reference( n"O", sio2 ),
    reference( n"Fe", fe ),
    reference( n"Ca", caf2 ),
    reference( n"Mg", mgo ),
], 132.0)

q20 = map(s->quantify(s, refs20), unk20)
ENV["columns"]=300
asa(DataFrame, q20, nominal = srm470_k412)

#### Example at 15 keV

In [ ]:
al2o3 = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "15 keV", "IIIE Al2O3[$i][4].msa") for i in 0:4)))
sio2 = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "15 keV", "IIIE SiO2[$i][4].msa") for i in 0:4)))
fe = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "15 keV", "IIIE Fe[$i][4].msa") for i in 0:4)))
caf2 = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "15 keV", "IIIE CaF2[$i][4].msa") for i in 0:4)))
mgo = sum(findsimilar(loadspectrum.(joinpath(path,"Multi-keV", "15 keV", "IIIE MgO[$i][4].msa") for i in 0:4)))
unk15 = loadspectrum.(joinpath(path,"Multi-keV", "15 keV", "IIIE K412[$i][4].msa") for i in 0:4)

refs15 = references( [
    reference( n"Al", al2o3 ),
    reference( n"Si", sio2 ),
    reference( n"O", sio2 ),
    reference( n"Fe", fe ),
    reference( n"Ca", caf2 ),
    reference( n"Mg", mgo ),
], 132.0)

q15 = map(s->quantify(s, refs15), unk15)
asa(DataFrame, q15, nominal=srm470_k412)

In [ ]:
asa(DataFrame, q15[1])

#### Mixed 15 keV unknown with 20 keV standards  

In [ ]:
q15_20 = map(s->quantify(s, refs20), unk15)
asa(DataFrame, q15_20, nominal = srm470_k412)

#### Mixed 20 keV unknown with 15 keV standards  

In [ ]:
q20_15 = map(s->quantify(s, refs15), unk20)
asa(DataFrame, q20_15, nominal=srm470_k412)

Let's consider what we have wrought...

Notice that the matrix correction factor has a new term, Generation, which accounts for the difference in ionization cross-section between the standard beam energy and the unknown.  The Z-, A- and F-terms is also different because the ϕ(ρz)-curve is also different.  The Z term as a significant difference because the character of the energy loss is very different. 

In [ ]:
asa(DataFrame, q20_15[1])

In [ ]:
asa(DataFrame, q15_20[1])

Finally, compare the mean estimated compositions.

In [ ]:
using Statistics
asa(DataFrame, [ mean(material.(q20)), mean(material.(q15)), mean(material.(q15_20)), mean(material.(q20_15)), srm470_k412 ])

O shows the largest deviations but the remaining elements look reasonably good regardless of choice of standards.

Q.E.D.